In [3]:
#bitmasks of fields 1-3, they fit in 2 bytes
FIELD1 = 0b1111000000000000 # first 4 bits
FIELD2 = 0b0000111000000000 # next 3 bits
FIELD3 = 0b0000000111111111 # last 9 bits


# --------------------------------------------------
# byte를 int로 변경 
def bytes_to_int(num):  #convert bytes object to an int
    res = 0
    num = num[::-1]  # reverse the bytes
    for i in range(len(num)):
        res += num[i] * (256**i)
    return res

# unpack
def get_fields(f):
    chunk = bytes_to_int(f.read(2))  # read 2 bytes, f1-f3, convert to int
    f1 = (chunk & FIELD1) >> 12  # get each field with its bitmask
    f2 = (chunk & FIELD2) >> 9
    f3 = chunk & FIELD3
    f4 = f.read(f3)  # field4 as a bytes object

    return f1, f2, f3, f4

def read_file():
    file = open('file.dat','rb')

    #using your sample data
    print(get_fields(file))  # returns 0, 4, 384, field4 as a bytes obj
    
    file.close()
    
# --------------------------------------------------
def int_to_byte(byte):  #convert int object to an byte
    res = 0
    byte = byte[::-1]  # reverse the bytes
    for i in range(len(byte)):
        res += byte[i] % (256**i)
    return res


def put_fields(filestream):
    chunk = bytes_to_int(f.read(2))  # read 2 bytes, f1-f3, convert to int
    f1 = (chunk & FIELD1) << 12  # get each field with its bitmask
    f2 = (chunk & FIELD2) << 9
    f3 = chunk & FIELD3
    f4 = f.read(f3)  # field4 as a bytes object

    return f1, f2, f3, f4
    
def write_file():
    newFileStream = [123, 3, 255, 0, 100]
    
    PackBytes = put_fields(newFileStream)
    # make file
    file = open("file.dat", "wb")
    # write to file
    for byte in PackBytes:
        file.write(byte.to_bytes(1, byteorder='big'))
        
    file.close()
    
write_file()

UnboundLocalError: local variable 'file' referenced before assignment